We try to classify if the human on the image carries a mask.
We use the dataset on https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset.

In [1]:
import tensorflow as tf

print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\Python39\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [ ]:
%ls
%cd face_mask_dataset

In [3]:
import pathlib

training_data_dir = pathlib.Path("Train")
validation_data_dir = pathlib.Path("Validation")
test_data_dir = pathlib.Path("Test")

In [4]:
batch_size = 32
img_width = 256
img_height = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  training_data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  validation_data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



Found 10000 files belonging to 2 classes.
Using 8000 files for training.
Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['WithMask', 'WithoutMask']


In [6]:
num_classes = 2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(img_height, img_width, 3)))
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 125, 125, 32)      4736      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 32)        50208     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 32)        50208     
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        50208     
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 3, 32)          50208     
                                                                 
Total params: 205,568
Trainable params: 205,568
Non-trai

In [7]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=3*3*32, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 125, 125, 32)      4736      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 32)        50208     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 32)        50208     
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        50208     
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 3, 32)          50208     
                                                                 
 flatten (Flatten)           (None, 288)               0

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=15)

Epoch 1/20
250/250 [==============================] - 30s 84ms/step - loss: 0.2866 - accuracy: 0.8900 - val_loss: 0.2160 - val_accuracy: 0.9312
Epoch 2/20
250/250 [==============================] - 21s 84ms/step - loss: 0.1694 - accuracy: 0.9374 - val_loss: 0.1372 - val_accuracy: 0.9438
Epoch 3/20
250/250 [==============================] - 21s 85ms/step - loss: 0.1434 - accuracy: 0.9488 - val_loss: 0.1044 - val_accuracy: 0.9438
Epoch 4/20
250/250 [==============================] - 23s 93ms/step - loss: 0.1029 - accuracy: 0.9620 - val_loss: 0.1306 - val_accuracy: 0.9500
Epoch 5/20
250/250 [==============================] - 21s 84ms/step - loss: 0.0896 - accuracy: 0.9678 - val_loss: 0.1166 - val_accuracy: 0.9625
Epoch 6/20
250/250 [==============================] - 22s 86ms/step - loss: 0.0876 - accuracy: 0.9672 - val_loss: 0.1044 - val_accuracy: 0.9750
Epoch 7/20
250/250 [==============================] - 24s 95ms/step - loss: 0.0708 - accuracy: 0.9728 - val_loss: 0.1006 - val_accuracy:

In [10]:
# Test
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data_dir,
  validation_split=None,
  subset=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

model.evaluate(test_ds)

Found 992 files belonging to 2 classes.
31/31 [==============================] - 2s 45ms/step - loss: 0.1396 - accuracy: 0.9607


[0.13958117365837097, 0.9606854915618896]